In [2]:
import numpy as np
import pandas as pd

In [38]:
data = pd.read_csv('/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/gemini_labeled_40k.csv')

In [39]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates,candidates_inx,hallucination_candidates_inx
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"['blue', 'hat']",['blue'],"{'blue': [(11, 15)], 'hat': [(4, 7)]}","{'blue': [(11, 15)]}"
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"['hat', 'gray']",['gray'],"{'hat': [(4, 7)], 'gray': [(11, 15)]}","{'gray': [(11, 15)]}"


In [29]:
data["candidates"] = data["candidates"].apply(lambda x: list(set(eval(x))))
data["hallucination_candidates"] = data["hallucination_candidates"].apply(lambda x: list(set(eval(x))))

In [30]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"[blue, hat]",[blue]
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"[hat, gray]",[gray]


In [2]:
import re
import unicodedata
from typing import Dict, Iterable, List, Tuple

def find_word_spans(
    text: str,
    terms: Iterable[str],
    *,
    case_sensitive: bool = False,
    whole_word: bool = False,
    overlapping: bool = False,
    normalize_unicode: bool = True,
) -> Dict[str, List[Tuple[int, int]]]:
    
    if normalize_unicode:
        text = unicodedata.normalize("NFC", text)

    # Prepare result with original (possibly unnormalized) term strings as keys
    results: Dict[str, List[Tuple[int, int]]] = {t: [] for t in terms if t}

    # Build a working list of (orig_term, norm_term)
    work_terms: List[Tuple[str, str]] = []
    for t in terms:
        if not t:
            continue
        t_norm = unicodedata.normalize("NFC", t) if normalize_unicode else t
        work_terms.append((t, t_norm))

    flags = 0 if case_sensitive else re.IGNORECASE

    for orig_term, norm_term in work_terms:
        # Escape to match literal text
        escaped = re.escape(norm_term)

        if whole_word:
            # Word boundaries only at the ends (so phrases are supported)
            # Use (?<!\w) and (?!\w) instead of \b to behave well inside lookaheads for overlapping.
            core = f"(?<!\\w){escaped}(?!\\w)"
        else:
            core = escaped

        if overlapping:
            # Lookahead to capture overlapping starts.
            # Put the core inside a capturing group so we can compute the end index.
            pattern = re.compile(f"(?=({core}))", flags)
            matches = pattern.finditer(text)
            for m in matches:
                start = m.start()
                end = start + len(m.group(1))
                results[orig_term].append((start, end))
        else:
            pattern = re.compile(core, flags)
            matches = pattern.finditer(text)
            for m in matches:
                results[orig_term].append((m.start(), m.end()))

    return results


In [32]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"[blue, hat]",[blue]
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"[hat, gray]",[gray]


In [33]:
candid_inx = [] 
hal_inx = []

for inx, row in data.iterrows():
    spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["candidates"]])
    candid_inx.append(spans)
    
    hal_spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["hallucination_candidates"]])
    hal_inx.append(hal_spans)
    

In [34]:
data["candidates_inx"] = candid_inx
data["hallucination_candidates_inx"] = hal_inx

In [35]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates,candidates_inx,hallucination_candidates_inx
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"[blue, hat]",[blue],"{'blue': [(11, 15)], 'hat': [(4, 7)]}","{'blue': [(11, 15)]}"
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"[hat, gray]",[gray],"{'hat': [(4, 7)], 'gray': [(11, 15)]}","{'gray': [(11, 15)]}"


In [36]:
data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/gemini_labeled_40k.csv", index=False)

In [58]:
sampled_data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/original_sampled_20k.csv")
total_data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/instruct/tp_train.csv")

In [59]:
sampled_ids = sampled_data["id"].to_list()

In [60]:
ramian_data = total_data[~total_data["id"].isin(sampled_ids)]

In [3]:
tp_data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/vqa/tp_train.csv")
tn_data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/vqa/tn_train.csv")

In [45]:
ramian_data = tn_data

In [46]:
ramian_data.head(5)

,Unnamed: 0,source_text,source_metadata,qa_metadata,qa_ids,prompt,image_id,hallucinated_text,annotations,split,id
0,40270,right,"{'id': '001000255', 'source': 'GQA'}",{},[],Use the provided image to answer the question:...,2411114,right,"{'object': [], 'attribute': [], 'relationship'...",train,vqa_1_clean
1,40271,post,"{'id': '001000300', 'source': 'GQA'}",{},[],What is the answer to the following question? ...,2411114,post,"{'object': [], 'attribute': [], 'relationship'...",train,vqa_2_clean
2,40272,left,"{'id': '001015025', 'source': 'GQA'}",{},[],Is the book to the right or to the left of the...,2387711,left,"{'object': [], 'attribute': [], 'relationship'...",train,vqa_4_clean
3,40273,book,"{'id': '001015051', 'source': 'GQA'}",{},[],What is the answer to the following question? ...,2387711,book,"{'object': [], 'attribute': [], 'relationship'...",train,vqa_5_clean
4,40274,blue,"{'id': '001017949', 'source': 'GQA'}",{},[],"Based on the image, respond to this question w...",2408027,blue,"{'object': [], 'attribute': [], 'relationship'...",train,vqa_6_clean


In [47]:
ramian_data["prompt"] = ramian_data["prompt"].apply(lambda x: x.replace("<image>",""))

In [48]:
ramian_data = ramian_data[["image_id", "prompt", "hallucinated_text", "annotations"]]

In [49]:
ramian_data.head(2)

,image_id,prompt,hallucinated_text,annotations
0,2411114,Use the provided image to answer the question:...,right,"{'object': [], 'attribute': [], 'relationship'..."
1,2411114,What is the answer to the following question? ...,post,"{'object': [], 'attribute': [], 'relationship'..."


In [50]:
all_hal_cand = []

for anot in ramian_data["annotations"]:
    res = eval(anot)
    hal_cand = []
    for val in res.values():
        for i in val:
            for j in list(i.values()):
                token =  j['name']
                hal_cand.append(token)
    all_hal_cand.append(list(set(hal_cand)))

In [51]:
ramian_data["hallucination_candidates"] = all_hal_cand
ramian_data["candidates"] = [[i] for i in ramian_data["hallucinated_text"]]


/tmp/ipykernel_3919724/1222014100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["hallucination_candidates"] = all_hal_cand
/tmp/ipykernel_3919724/1222014100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["candidates"] = [[i] for i in ramian_data["hallucinated_text"]]


In [52]:
ramian_data.drop(columns=["annotations"], inplace=True)

/tmp/ipykernel_3919724/643254796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data.drop(columns=["annotations"], inplace=True)


In [53]:
ramian_data.head(2)

,image_id,prompt,hallucinated_text,hallucination_candidates,candidates
0,2411114,Use the provided image to answer the question:...,right,[],[right]
1,2411114,What is the answer to the following question? ...,post,[],[post]


In [54]:
ramian_data.columns = ["image_path", "question", "answer", "hallucination_candidates", "candidates"]


In [55]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates
0,2411114,Use the provided image to answer the question:...,right,[],[right]
1,2411114,What is the answer to the following question? ...,post,[],[post]


In [57]:
hal_inx = []

for inx, row in ramian_data.iterrows():
    hal_spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["candidates"]])
    hal_inx.append(hal_spans)
    

In [58]:
ramian_data["candidates_inx"] = hal_inx

/tmp/ipykernel_3919724/4104138947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["candidates_inx"] = hal_inx


In [59]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates,candidates_inx
0,2411114,Use the provided image to answer the question:...,right,[],[right],"{'right': [(0, 5)]}"
1,2411114,What is the answer to the following question? ...,post,[],[post],"{'post': [(0, 4)]}"


In [60]:
ramian_data["image_path"] = ramian_data["image_path"].apply(lambda x: str(x)+ ".jpg")

/tmp/ipykernel_3919724/308830470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["image_path"] = ramian_data["image_path"].apply(lambda x: str(x)+ ".jpg")


In [61]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates,candidates_inx
0,2411114.jpg,Use the provided image to answer the question:...,right,[],[right],"{'right': [(0, 5)]}"
1,2411114.jpg,What is the answer to the following question? ...,post,[],[post],"{'post': [(0, 4)]}"


In [62]:
ramian_data.shape

(40270, 6)

In [63]:
ramian_data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/vqa/tn_data.csv", index=False)

In [22]:
data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/caption/gemini_labeled_28k.csv")

In [29]:
tar_paths = ["/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/caption/gemini_labeled_28k.csv",
             "/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/caption/original_unsampled_14k.csv",
             "/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/gemini_labeled_40k.csv",
             "/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/original_unsampled_20k.csv",
             "/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/vqa/tn_data.csv",
             "/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/vqa/tp_data.csv"
             ]

In [30]:
from uuid import uuid4

for path in tar_paths:
    data = pd.read_csv(path)
    img_folder = "/Data2/Arun-UAV/NLP/vision_halu/visual_genome/target_images/"

    data = data.rename(columns={"image_path": "image_id"})
    data["image_path"] = data["image_id"].apply(lambda x: img_folder + x)
    data["question_id"] = [str(uuid4()) for _ in range(len(data))]
    
    data.to_csv(path, index=False)

In [44]:
pd.read_csv(tar_paths[0]).head(2)

,image_id,question,answer,candidates,hallucination_candidates,candidates_inx,hallucination_candidates_inx,image_path,question_id
0,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"['holding', 'playing', 'fencing', 'tennis', 'b...",['bat'],"{'holding': [(40, 47)], 'playing': [(58, 65)],...","{'bat': [(50, 53)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,10850c51-bdb5-4a1d-9739-a72f955e84a7
1,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"['ground', 'holding', 'playing', 'fencing', 't...","['frisbee', 'bat']","{'ground': [(114, 120)], 'holding': [(40, 47)]...","{'frisbee': [(99, 106)], 'bat': [(50, 53)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,78581d22-2e82-439a-987f-2504e3642904


In [1]:
(4096*690*16*2/1024**3)

0.084228515625

In [6]:
text_1 == text_2

True